In [1]:
#mount drive
from google.colab import drive
#drive.mount('/content/gdrive')

drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#switch to gpu
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
import torch, os
import sys

drg_repo_path = '/content/gdrive/My Drive/humor_style_transfer/transformer-drg-style-transfer'
if not drg_repo_path in sys.path:
  sys.path.append(drg_repo_path)

from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from tqdm import tqdm
import numpy as np
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


## Delete and Generate

In [0]:
special_tokens = ['<POS>', '<NEG>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
### which model is this???
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

100%|██████████| 458495/458495 [00:00<00:00, 990407.55B/s]
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
100%|██████████| 656/656 [00:00<00:00, 143702.06B/s]


In [0]:
## [CUSTOMIZE]: change to whatever GPT model path we just trained.
jokes_gpt_model_path = '/content/gdrive/My Drive/humor_style_transfer/transformer-drg-style-transfer/reddit_jokes_scripts/dg_model/'
path = os.path.join(jokes_gpt_model_path,'pytorch_model_zero_grad_1.bin') ## Model Path
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40483, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout

In [0]:
## [CUSTOMIZE]: change to reddit jokes bert classifier
bert_classifier_dir = "/content/gdrive/My Drive/humor_style_transfer/transformer-drg-style-transfer/reddit_jokes_scripts/bert_classifier" 
model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_dir, num_labels=2)
tokenizer_cls = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()

100%|██████████| 231508/231508 [00:00<00:00, 830086.21B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

In [0]:
#manually changed to 40 from 70...
max_seq_len=40
sm = torch.nn.Softmax(dim=-1)

In [0]:
model.config.n_positions

512

In [0]:
def prediction_with_beam_search(ref_text, beam_width=3, vocab_length=40483):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    #index_tokens = [indexed_tokens for i in range(beam_width)]
    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    while count < model.config.n_positions and not stop_decode:
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                preds = sm(model(torch_tensor)) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += 1
        else: # After first step
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            # Prediction on the current state
            with torch.no_grad():
                preds = sm(model(current_state))
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            # Fatch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == tokenizer.special_tokens["<END>"]:
                    done[i] = True
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[x] + [correct_inx[i]]
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(tokenizer.special_tokens["<END>"])
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences

In [0]:
def get_best_sentence(input_sentences, sentiment=1):
    """
    This function selects the sentence from the Beam of the sentences,
    based on the classification probability score.
    
    input_sentences : list of strings : Sentences generated by the Beam search decoding
    sentiment: int : Expected sentiment (in general class for the classification)
    """
    # BERT pre-processing
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer_cls.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer_cls.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    ids = torch.tensor(ids).to(device)
    segment_ids = torch.tensor(segment_ids).to(device)
    input_masks = torch.tensor(input_masks).to(device)
    # prediction
    with torch.no_grad():
        preds = sm(model_cls(ids, segment_ids, input_masks))
        
    preds = preds.tolist()
    inx, inx_val = None, 0
    for i in range(len(input_sentences)):
        temp = preds[i][sentiment]
        if temp > inx_val:
            inx = i
            inx_val = temp
    return input_sentences[inx]

## Example

In [0]:
#here, we're trying to transform this sentence with the joke style (it works reasonably well!)
op=prediction_with_beam_search("<POS> <CON_START> it is not terrible , but it is very good . <START>",4)
op

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


['it is not terrible , but it is very bad . " 1',
 'it is not terrible , but it is very good . " 1',
 'it is not terrible , but it is very bad . 1',
 'it is not terrible , but it is very bad news . " 1']

In [0]:
get_best_sentence(op)

'it is not terrible , but it is very bad news . " 1'

### Predictions for the reference files

In [0]:
'''
  TODO: for this next part...
  "version 1.0": we have to add in the captions in accordance with the style above
    - start by not deleting any words
    - ex. "<POS> <CON_START> child in a pink dress is climbing up a set of stairs in an entry way . <START>"
  
  "version 2.0": I think pass the sentence through the "delete" pipeline first?

  "version 3.0": Add in retrieval
  "version 4.0": Add in retrieval, and add subject swapping?
'''

In [0]:
#[CUSTOMIZE]: add directory for attention-preprocessed files.
output_dir = '/content/gdrive/My Drive/humor_style_transfer/reddit_jokes/joke-dataset/style_trans_preprocessing/processed_files_with_bert_with_best_head/'
#output_dir = "/home/ubuntu/bhargav/data/yelp/processed_files_with_bert_with_best_head/"
os.listdir(output_dir)

['jokes.train.0',
 'jokes.train.1',
 'jokes.dev.0',
 'jokes.dev.1',
 'jokes.test.1',
 'jokes.test.0',
 'flickr.reference.1.txt',
 'flickr_v1_reference_1_predictions_with_beam_search.txt']

In [0]:
%ls '/content/gdrive/My Drive/humor_style_transfer/reddit_jokes/joke-dataset/style_trans_preprocessing/processed_files_with_bert_with_best_head/'

ls: cannot access '/content/gdrive/My Drive/humor_style_transfer/reddit_jokes/joke-dataset/style_trans_preprocessing/processed_files_with_bert_with_best_head/': No such file or directory


In [0]:
#[TODO]: filter out sequences longer than 40 characters. Otherwise the below will crash
#[CUSTOMIZE]: change input / output filename
#only outputting 1000 examples. Don't think we can really analyze more than that anyway...
ref_file_name = './flickr.reference.1.txt'
out_file_name = './flickr_v1_reference_1_predictions_with_beam_search.txt'
n_outputs = 1000
with open(os.path.join(output_dir, out_file_name) ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, ref_file_name)) as fp:
        for line in fp:
            out_sen = prediction_with_beam_search(line.strip(), 5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            ## todo: potentially filter out too-long answers.
            print(c,get_best_sentence(out_sen, sentiment = 1)) # <<< target style is 1 (funny) ... not changing anything :(
            c += 1
            if c >= 1000:
              break
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 child in a pink dress is climbing up a set of stairs in an entry way . " 1
1 little girl going into a wooden building . " 1
2 little girl climbing into a wooden playhouse . " 1
3 little girl climbing the stairs to her playhouse . " 1
4 little girl in a pink dress going into a wooden cabin . " 1
5 black dog and a spotted dog are inseparable . 1
6 black dog and a tri - colored dog playing with each other on the road . " 1
7 black dog and a white dog are staring at each other in the street . " 1
8 dogs of different breeds looking at each other on the road . " 1
9 dogs on sidewalks moving toward each other . " 1
10 little girl covered in paint sits in front of a painted rainbow with her hands in a bowl . " 1
11 little girl is sitting in front of a large painting . " 1
12 small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow painted on it . " 1
13 is a girl with pigtails sitting in front of a rainbow painting . " 1
14 little girl with pigtails painting 

In [0]:
# We can skip this one. Style-0 is bad jokes, so it will only generate bad jokes (not useful).
ref_file_name = './flickr.reference.1.txt'
out_file_name = './flickr_v1_reference_1_predictions_with_beam_search.txt'
with open(os.path.join(output_dir, out_file_name) ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, ref_file_name)) as fp:
        for line in fp:
            #def prediction_with_beam_search(ref_text, beam_width=3, vocab_length=40483):
            out_sen = prediction_with_beam_search(line.strip(), 5, max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))  ## <<< the target style here is "0"
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

0 child in a pink dress is climbing up a set of stairs in an entrance way . 1
1 damn girl going into a wooden building . 1
2 little girl climbing into a wooden playhouse . 1
3 little girl climbing the stairs to her playhouse . 1
4 little girl in a pink dress going into a wood cabin . " 1
5 black dog and a spotted dog are . " 1
6 black dog and a tri - colored dog playing together on the road . " 1
7 black dog and a white dog with brown spots are staring at each other in the street . 1
8 dogs of different breeds looking at each other on the road . woof 1
9 dogs on asphalt moving toward each other . " 1
10 little girl covered in paint sits in front of a painted rainbow with her hands in a bowl . 1
11 little girl is sitting in front of a large paint rainbow . " 1
12 small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on it . 1
13 barbie is a girl with pigtails sitting in front of a rainbow painting . 1
14 damn girl with pigtails painting outside in the

ValueError: ignored

TODO NEXT: 
* the above code went through and generated "funny" styled captions using Delete + Retrieve.
* We need to also prep data + models and run the "retrieve" step (below).

## Delete, Retrieve and Generate

In [4]:
special_tokens = ['<ATTR_WORDS>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

100%|██████████| 458495/458495 [00:00<00:00, 8180306.75B/s]
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
100%|██████████| 656/656 [00:00<00:00, 229690.58B/s]


In [5]:
# [CUSTOMIZE]: add the recently trained DRG model
jokes_drg_model_path = '/content/gdrive/My Drive/humor_style_transfer/transformer-drg-style-transfer/reddit_jokes_scripts/drg_model/'
path = os.path.join(jokes_drg_model_path, "./pytorch_model_zero_grad_1.bin")
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40482, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout

In [7]:
# [CUSTOMIZE]: choose dir with tfidf ref files.
# [NOTE]: this means we must redo the "tfidf" step! The "reference" files have to contain captions matched w joke content.
# Output dir have reference files generated using TFIDF for retrieve attributes from opposite corpus

#output_dir = "/home/ubuntu/bhargav/data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"
output_dir = '/content/gdrive/My Drive/humor_style_transfer/reddit_jokes/joke-dataset/style_trans_preprocessing/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/'
os.listdir(output_dir)

['reference_1.txt', 'reference_0.txt']

In [0]:
with open(os.path.join(output_dir,"reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_0_content.txt")) as fp:
        for line in fp:
            out_sen = prediction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

In [0]:
with open(os.path.join(output_dir,"reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_1_content.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")